In [ ]:
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 KB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 KB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 KB 20.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.6/99.6 KB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 57.8 MB/s eta 0:00:00
  Preparing metadata

In [ ]:
import ee
import geemap

In [ ]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=Zn0pi2-Olwuerujv5uD5gZZzJG048GJlZiGciNZjdLA&tc=xx9pAcozLFL5143nAQMhWZhFrhlLf2uMbAhxs1jflNM&cc=KeysshIHPwjrgem9xG0sHnr4QZxaKyPCXJO_QEBPlig

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh7XQ_94TfdVXkC5C7HEwzhrDw3N2QjN3iPHEpQgttY7VcD857IDL70

Successfully saved authorization token.


In [ ]:
geometry =ee.Geometry.Polygon(
        [[[77.39980020154476, 23.209794968246],
          [77.40042247403622, 23.209794968246],
          [77.4008516274786, 23.209755526121455],
          [77.40121640790463, 23.209479430923615],
          [77.40158118833065, 23.209696362912805],
          [77.40143098462582, 23.210011899722915],
          [77.40098037351132, 23.210189388851145],
          [77.40035810101986, 23.210149946843007],
          [77.40115203488827, 23.210603529233456],
          [77.40093745816708, 23.210938784793257],
          [77.40100183118344, 23.21139236450578],
          [77.4014095269537, 23.211767059628617],
          [77.40100183118344, 23.211688176532185],
          [77.40025081265927, 23.211806501159376],
          [77.40014352429867, 23.21147124777687],
          [77.39992894757748, 23.21115571441311],
          [77.39962854016781, 23.210781017575787],
          [77.39945687879086, 23.210386598717097],
          [77.39930667508602, 23.20989357350644]]]);

In [ ]:
import numpy as np
import pandas as pd
Start_date = '2020-01-01'
End_date = '2021-12-31'
bands = ['B3','B8']
dataset = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                  .filterBounds(geometry) \
                  .filterDate(Start_date,End_date) \
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)) \
                  .median() 
ndwi = dataset.normalizedDifference(bands).rename('ndwi')

latlon = ee.Image.pixelLonLat().addBands(ndwi)

# apply reducer to list
latlon = latlon.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=geometry,
      scale=30)
data_ndwi = np.array((ee.Array(latlon.get('ndwi')).getInfo()))

band_ndci = ['B4','B3']
band_do=['B4','B5']
band_temp=['B10']
band_ndti = ['B8','B12']
band_ndsi = ['B3','B11']
dataset = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                  .filterBounds(geometry) \
                  .filterDate(Start_date,End_date) \
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)) \
                  .median() 
ndci = dataset.normalizedDifference(bands).rename('ndci')
latlon = ee.Image.pixelLonLat().addBands(ndci)
latlon = latlon.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=geometry,
      scale=30)
assert len(band_ndci) == 2, 'bands_ndci should contain exactly 2 band names'
data_ndci = np.array((ee.Array(latlon.get('ndci')).getInfo()))

dissolveoxygen = dataset.normalizedDifference(band_do).rename('dissolveoxygen')
latlon = ee.Image.pixelLonLat().addBands(dissolveoxygen)
latlon = latlon.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=geometry,
      scale=30,
      tileScale = 16)
data_do = np.array((ee.Array(latlon.get('dissolveoxygen')).getInfo()))

temp = dataset.select(band_temp).rename('temp')
latlon = ee.Image.pixelLonLat().addBands(temp)
latlon = latlon.reduceRegion(
        reducer=ee.Reducer.toList(),
        geometry=geometry,
        scale=30)
ndti =dataset.normalizedDifference(bands).rename('ndti')
latlon = ee.Image.pixelLonLat().addBands(ndti)
latlon = latlon.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=geometry,
      scale=30)
data_ndti = np.array((ee.Array(latlon.get('ndti')).getInfo()))

ndsi=dataset.normalizedDifference(bands).rename('ndsi')
latlon = ee.Image.pixelLonLat().addBands(ndsi)
# apply reducer to list
latlon = latlon.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=geometry,
      scale=30)
data_ndsi = np.array((ee.Array(latlon.get('ndsi')).getInfo()))

data = pd.DataFrame({'NDWI': data_ndwi,
                     'NDCI': data_ndci,
                     'NDTI':data_ndti,
                     'DO':data_do,
                     'NDSI':data_ndsi})
data.head()

,NDWI,NDCI,NDTI,DO,NDSI
0,-0.255780,-0.255780,-0.255780,-0.150908,-0.255780
1,-0.367697,-0.367697,-0.367697,-0.146540,-0.367697
2,-0.422582,-0.422582,-0.422582,-0.134282,-0.422582
3,-0.415623,-0.415623,-0.415623,-0.117402,-0.415623
4,-0.388830,-0.388830,-0.388830,-0.129893,-0.388830


In [1]:
'''Map.addLayer(sentinel, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'RGB')
Map.addLayer(ndwi, {'min': -1, 'max': 1, 'palette': ['white', 'black', 'blue']}, 'NDWI')
Map.centerObject(poi, zoom=15)
Map'''

"Map.addLayer(sentinel, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'RGB')\nMap.addLayer(ndwi, {'min': -1, 'max': 1, 'palette': ['white', 'black', 'blue']}, 'NDWI')\nMap.centerObject(poi, zoom=15)\nMap"